In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random

import numpy as np
import tensorflow as tf

from helpers.dataset import TensorFlowDataset, import_zip_file
from helpers.evaluate import TensorFlowModelEvaluator
from models.rnn import LSTM
from vanilla_neural_nets.recurrent_neural_network.training_data import WordLevelRNNTrainingDataBuilder

# Create some data

In [3]:
PATH = '../data/text8.zip'
N_CLASSES = VOCABULARY_SIZE = 1000
BATCH_SIZE = 128
TIME_STEPS = 10
TRAINING_EXAMPLES = BATCH_SIZE * 50

In [4]:
corpus = import_zip_file(path=PATH, n_characters=1000000)

In [5]:
training_data = WordLevelRNNTrainingDataBuilder.build(corpus=corpus, vocabulary_size=VOCABULARY_SIZE)
training_data = np.array(training_data.training_data_as_indices[0][:TRAINING_EXAMPLES*(TIME_STEPS+1)])
training_data = training_data.reshape(TRAINING_EXAMPLES, TIME_STEPS+1)

In [6]:
training_dataset = TensorFlowDataset(
    data=training_data[:BATCH_SIZE*48, :TIME_STEPS], 
    labels=training_data[:BATCH_SIZE*48, 1:]
)

validation_dataset = TensorFlowDataset(
    data=training_data[BATCH_SIZE*48:BATCH_SIZE*49, :TIME_STEPS], 
    labels=training_data[BATCH_SIZE*48:BATCH_SIZE*49, 1:]
)

test_dataset = TensorFlowDataset(
    data=training_data[BATCH_SIZE*49:, :TIME_STEPS], 
    labels=training_data[BATCH_SIZE*49:, 1:]
)

In [7]:
EMBEDDING_LAYER_SIZE = 128
HIDDEN_STATE_SIZE = 100
LEARNING_RATE = .1
N_EPOCHS = int(1e8)

# Define graph

In [8]:
graph = tf.Graph()

with graph.as_default():
    
    dataset = TensorFlowDataset(
        data=tf.placeholder(dtype=tf.int32, shape=[BATCH_SIZE, TIME_STEPS]),
        labels=tf.placeholder(dtype=tf.int32, shape=[BATCH_SIZE, TIME_STEPS])
    )
    
    model = LSTM(
        dataset=dataset, 
        n_classes=N_CLASSES,
        embedding_layer_size=EMBEDDING_LAYER_SIZE,
        hidden_state_size=HIDDEN_STATE_SIZE,
        learning_rate=LEARNING_RATE
    )

In [9]:
with tf.Session(graph=graph) as session:

    session.run(tf.initialize_all_variables())

    evaluator = TensorFlowModelEvaluator(
        model=model,
        session=session,
        validation_dataset=validation_dataset,
        test_dataset=test_dataset
    )

    for epoch in range(N_EPOCHS):
        
        if epoch % int(1e7) == 0:

            mini_batch_data, mini_batch_labels = training_dataset.sample(BATCH_SIZE)
            mini_batch_dataset = TensorFlowDataset(data=mini_batch_data, labels=mini_batch_labels)
            evaluator.optimize(mini_batch_dataset)

            print('Epoch: {}'.format( int(epoch / 1e7) ))
            print('Train Loss: {:.3f}'.format(evaluator.training_loss))
            print('Validation Loss: {:.3f}\n'.format(evaluator.validation_loss))

    print('Test Loss: {0:.3f}\n'.format(evaluator.test_loss))

Epoch: 0
Train Loss: 7.368
Validation Loss: 7.340

Epoch: 1
Train Loss: 7.218
Validation Loss: 7.244

Epoch: 2
Train Loss: 7.135
Validation Loss: 7.135

Epoch: 3
Train Loss: 7.008
Validation Loss: 7.026

Epoch: 4
Train Loss: 6.914
Validation Loss: 6.900

Epoch: 5
Train Loss: 6.797
Validation Loss: 6.737

Epoch: 6
Train Loss: 6.528
Validation Loss: 6.535

Epoch: 7
Train Loss: 6.352
Validation Loss: 6.319

Epoch: 8
Train Loss: 6.085
Validation Loss: 6.167

Epoch: 9
Train Loss: 5.875
Validation Loss: 6.086

Test Loss: 5.880

